In [1]:
# default_exp models.tacotron2

In [1]:
# export
from torch import nn
from uberduck_ml_dev.models.base import TTSModel
from uberduck_ml_dev.models.common import Attention, Conv1d, LinearNorm, GST
from uberduck_ml_dev.text.symbols import symbols
from uberduck_ml_dev.vendor.tfcompat.hparam import HParams
from uberduck_ml_dev.utils.utils import to_gpu, get_mask_from_lengths
import numpy as np
import torch
from torch.autograd import Variable
from torch.cuda.amp import autocast
from torch.nn import functional as F

# mix
# import pdb

# class Decoder(nn.Module):
#     def __init__(self, hparams):
#         super().__init__()
#         self.n_mel_channels = hparams.n_mel_channels
#         self.n_frames_per_step_initial = hparams.n_frames_per_step_initial
#         self.n_frames_per_step_current = hparams.n_frames_per_step_initial
#         self.encoder_embedding_dim = hparams.encoder_embedding_dim
#         self.attention_rnn_dim = hparams.attention_rnn_dim
#         self.decoder_rnn_dim = hparams.decoder_rnn_dim
#         self.prenet_dim = hparams.prenet_dim
#         self.max_decoder_steps = hparams.max_decoder_steps
#         self.gate_threshold = hparams.gate_threshold
#         self.p_attention_dropout = hparams.p_attention_dropout
#         self.p_decoder_dropout = hparams.p_decoder_dropout
#         self.p_teacher_forcing = hparams.p_teacher_forcing

#         self.prenet = Prenet(
#             hparams.n_mel_channels,
#             [hparams.prenet_dim, hparams.prenet_dim],
#         )

#         self.attention_rnn = nn.LSTMCell(
#             hparams.prenet_dim + self.encoder_embedding_dim,
#             hparams.attention_rnn_dim,
#         )

#         self.attention_layer = Attention(
#             hparams.attention_rnn_dim,
#             self.encoder_embedding_dim,
#             hparams.attention_dim,
#             hparams.attention_location_n_filters,
#             hparams.attention_location_kernel_size,
#             fp16_run=hparams.fp16_run,
#         )

#         self.decoder_rnn = nn.LSTMCell(
#             hparams.attention_rnn_dim + self.encoder_embedding_dim,
#             hparams.decoder_rnn_dim,
#             1,
#         )

#         self.linear_projection = LinearNorm(
#             hparams.decoder_rnn_dim + self.encoder_embedding_dim,
#             hparams.n_mel_channels * hparams.n_frames_per_step_initial,
#         )

#         self.gate_layer = LinearNorm(
#             hparams.decoder_rnn_dim + self.encoder_embedding_dim,
#             1,
#             bias=True,
#             w_init_gain="sigmoid",
#         )

#     def set_current_frames_per_step(self, n_frames: int):
#         self.n_frames_per_step_current = n_frames

#     def get_go_frame(self, memory):
#         """Gets all zeros frames to use as first decoder input
#         PARAMS
#         ------
#         memory: decoder outputs

#         RETURNS
#         -------
#         decoder_input: all zeros frames
#         """
#         B = memory.size(0)
#         decoder_input = Variable(memory.data.new(B, self.n_mel_channels).zero_())
#         return decoder_input

#     def initialize_decoder_states(self, memory, mask):
#         """Initializes attention rnn states, decoder rnn states, attention
#         weights, attention cumulative weights, attention context, stores memory
#         and stores processed memory
#         PARAMS
#         ------
#         memory: Encoder outputs
#         mask: Mask for padded data if training, expects None for inference
#         """
#         B = memory.size(0)
#         MAX_TIME = memory.size(1)

#         self.attention_hidden = Variable(
#             memory.data.new(B, self.attention_rnn_dim).zero_()
#         )
#         self.attention_cell = Variable(
#             memory.data.new(B, self.attention_rnn_dim).zero_()
#         )

#         self.decoder_hidden = Variable(memory.data.new(B, self.decoder_rnn_dim).zero_())
#         self.decoder_cell = Variable(memory.data.new(B, self.decoder_rnn_dim).zero_())

#         self.attention_weights = Variable(memory.data.new(B, MAX_TIME).zero_())
#         self.attention_weights_cum = Variable(memory.data.new(B, MAX_TIME).zero_())
#         self.attention_context = Variable(
#             memory.data.new(B, self.encoder_embedding_dim).zero_()
#         )

#         self.memory = memory
#         self.processed_memory = self.attention_layer.memory_layer(memory)
#         self.mask = mask
# #     def initialize_decoder_states(self, memory):
# #         """Initializes attention rnn states, decoder rnn states, attention
# #         weights, attention cumulative weights, attention context, stores memory
# #         and stores processed memory
# #         PARAMS
# #         ------
# #         memory: Encoder outputs
# #         mask: Mask for padded data if training, expects None for inference
# #         """
# #         B = memory.size(0)
# #         MAX_TIME = memory.size(1)
# #         dtype = memory.dtype
# #         device = memory.device

# #         attention_hidden = torch.zeros(
# #             B, self.attention_rnn_dim, dtype=dtype, device=device
# #         )
# #         attention_cell = torch.zeros(
# #             B, self.attention_rnn_dim, dtype=dtype, device=device
# #         )

# #         decoder_hidden = torch.zeros(
# #             B, self.decoder_rnn_dim, dtype=dtype, device=device
# #         )
# #         decoder_cell = torch.zeros(B, self.decoder_rnn_dim, dtype=dtype, device=device)

# #         attention_weights = torch.zeros(B, MAX_TIME, dtype=dtype, device=device)
# #         attention_weights_cum = torch.zeros(B, MAX_TIME, dtype=dtype, device=device)
# #         attention_context = torch.zeros(
# #             B, self.encoder_embedding_dim, dtype=dtype, device=device
# #         )

# #         processed_memory = self.attention_layer.memory_layer(memory)

# #         return (
# #             attention_hidden,
# #             attention_cell,
# #             decoder_hidden,
# #             decoder_cell,
# #             attention_weights,
# #             attention_weights_cum,
# #             attention_context,
# #             processed_memory,
# #         )

#     def parse_decoder_inputs(self, decoder_inputs):
#         """Prepares decoder inputs, i.e. mel outputs
#         PARAMS
#         ------
#         decoder_inputs: inputs used for teacher-forced training, i.e. mel-specs

#         RETURNS
#         -------
#         inputs: processed decoder inputs

#         """
#         # (B, n_mel_channels, T_out) -> (B, T_out, n_mel_channels)

#         decoder_inputs = decoder_inputs.transpose(1, 2)
#         decoder_inputs = decoder_inputs.contiguous()
#         # print(decoder_inputs.shape, 'dec_in')
#         # print(decoder_inputs.size(0), int(decoder_inputs.size(1)), self.n_frames_per_step_current)
#         decoder_inputs = decoder_inputs.view(
#             decoder_inputs.size(0),
#             int(decoder_inputs.size(1) / self.n_frames_per_step_current),
#             -1,
#         )
#         # (B, T_out, n_mel_channels) -> (T_out, B, n_mel_channels)
#         decoder_inputs = decoder_inputs.transpose(0, 1)
#         return decoder_inputs

#     def parse_decoder_outputs(self, mel_outputs, gate_outputs, alignments):
#         """Prepares decoder outputs for output
#         PARAMS
#         ------
#         mel_outputs:
#         gate_outputs: gate output energies
#         alignments:

#         RETURNS
#         -------
#         mel_outputs:
#         gate_outpust: gate output energies
#         alignments:
#         """
#         # (T_out, B) -> (B, T_out)
#         alignments = torch.stack(alignments).transpose(0, 1)
#         # (T_out, B) -> (B, T_out)
#         gate_outputs = torch.stack(gate_outputs)
#         if len(gate_outputs.size()) > 1:
#             gate_outputs = gate_outputs.transpose(0, 1)
#         else:
#             gate_outputs = gate_outputs[None]
#         gate_outputs = gate_outputs.contiguous()
#         # (B, T_out, n_mel_channels * n_frames_per_step) -> (B, T_out * n_frames_per_step, n_mel_channels)
#         mel_outputs = mel_outputs.view(mel_outputs.size(0), -1, self.n_mel_channels)
#         # (B, T_out, n_mel_channels) -> (B, n_mel_channels, T_out)
#         mel_outputs = mel_outputs.transpose(1, 2)

#         return mel_outputs, gate_outputs, alignments

#     def decode(self, decoder_input, attention_weights=None):
#         """Decoder step using stored states, attention and memory
#         PARAMS
#         ------
#         decoder_input: previous mel output

#         RETURNS
#         -------
#         mel_output:
#         gate_output: gate output energies
#         attention_weights:
#         """
#         cell_input = torch.cat((decoder_input, self.attention_context), -1)
#         self.attention_hidden, self.attention_cell = self.attention_rnn(
#             cell_input, (self.attention_hidden, self.attention_cell)
#         )
#         self.attention_hidden = F.dropout(
#             self.attention_hidden, self.p_attention_dropout, self.training
#         )
#         self.attention_cell = F.dropout(
#             self.attention_cell, self.p_attention_dropout, self.training
#         )

#         attention_weights_cat = torch.cat(
#             (
#                 self.attention_weights.unsqueeze(1),
#                 self.attention_weights_cum.unsqueeze(1),
#             ),
#             dim=1,
#         )
#         self.attention_context, self.attention_weights = self.attention_layer(
#             self.attention_hidden,
#             self.memory,
#             self.processed_memory,
#             attention_weights_cat,
#             self.mask,
#             attention_weights,
#         )

#         self.attention_weights_cum += self.attention_weights
#         decoder_input = torch.cat((self.attention_hidden, self.attention_context), -1)
#         self.decoder_hidden, self.decoder_cell = self.decoder_rnn(
#             decoder_input, (self.decoder_hidden, self.decoder_cell)
#         )
#         self.decoder_hidden = F.dropout(
#             self.decoder_hidden, self.p_decoder_dropout, self.training
#         )
#         self.decoder_cell = F.dropout(
#             self.decoder_cell, self.p_decoder_dropout, self.training
#         )

#         decoder_hidden_attention_context = torch.cat(
#             (self.decoder_hidden, self.attention_context), dim=1
#         )

#         decoder_output = self.linear_projection(decoder_hidden_attention_context)

#         gate_prediction = self.gate_layer(decoder_hidden_attention_context)
#         return decoder_output, gate_prediction, self.attention_weights

#     def forward(self, memory, decoder_inputs, memory_lengths):
#         """Decoder forward pass for training
#         PARAMS
#         ------
#         memory: Encoder outputs
#         decoder_inputs: Decoder inputs for teacher forcing. i.e. mel-specs
#         memory_lengths: Encoder output lengths for attention masking.

#         RETURNS
#         -------
#         mel_outputs: mel outputs from the decoder
#         gate_outputs: gate outputs from the decoder
#         alignments: sequence of attention weights from the decoder
#         """
#         B = memory.size(0)
#         decoder_inputs = self.parse_decoder_inputs(decoder_inputs)
#         decoder_inputs = decoder_inputs.reshape(
#             -1, decoder_inputs.size(1), self.n_mel_channels
#         )
#         decoder_input = self.get_go_frame(memory).unsqueeze(0)
#         decoder_inputs = torch.cat((decoder_input, decoder_inputs), dim=0)
#         decoder_inputs = self.prenet(decoder_inputs)

#         self.initialize_decoder_states(
#             memory, mask=~get_mask_from_lengths(memory_lengths)
#         )

#         mel_outputs = torch.empty(
#             B, 0, self.n_frames_per_step_current * self.n_mel_channels
#         )
#         if torch.cuda.is_available():
#             mel_outputs = mel_outputs.cuda()
#         gate_outputs, alignments = [], []
#         desired_output_frames = decoder_inputs.size(0) / self.n_frames_per_step_current
#         while mel_outputs.size(1) < desired_output_frames - 1:
#             if (
#                 mel_outputs.size(1) == 0
#                 or np.random.uniform(0.0, 1.0) <= self.p_teacher_forcing
#             ):
#                 teacher_forced_frame = decoder_inputs[
#                     mel_outputs.size(1) * self.n_frames_per_step_current
#                 ]

#                 to_concat = (teacher_forced_frame,)
#                 decoder_input = torch.cat(to_concat, dim=1)
#             else:

#                 # NOTE(zach): we may need to concat these as we go to ensure that
#                 # it's easy to retrieve the last n_frames_per_step_init frames.
#                 to_concat = (
#                     self.prenet(
#                         mel_outputs[:, -1, -1 * self.n_frames_per_step_current :]
#                     ),
#                 )
#                 decoder_input = torch.cat(to_concat, dim=1)
#             # NOTE(zach): When training with fp16_run == True, decoder_rnn seems to run into
#             # issues with NaNs in gradient, maybe due to vanishing gradients.
#             # Disable half-precision for this call to work around the issue.
#             with autocast(enabled=False):
#                 mel_output, gate_output, attention_weights = self.decode(decoder_input)
#             mel_outputs = torch.cat(
#                 [
#                     mel_outputs,
#                     mel_output[
#                         :, 0 : self.n_mel_channels * self.n_frames_per_step_current
#                     ].unsqueeze(1),
#                 ],
#                 dim=1,
#             )
#             gate_outputs += [gate_output.squeeze()] * self.n_frames_per_step_current
#             alignments += [attention_weights]

#         mel_outputs, gate_outputs, alignments = self.parse_decoder_outputs(
#             mel_outputs, gate_outputs, alignments
#         )

#         return mel_outputs, gate_outputs, alignments

#     def inference(self, memory, memory_lengths):
#         """Decoder inference
#         PARAMS
#         ------
#         memory: Encoder outputs
#         RETURNS
#         -------
#         mel_outputs: mel outputs from the decoder
#         gate_outputs: gate outputs from the decoder
#         alignments: sequence of attention weights from the decoder
#         """
#         decoder_input = self.get_go_frame(memory)
#         pdb.set_trace()
#         mask = get_mask_from_lengths(memory_lengths)
#         (
#             attention_hidden,
#             attention_cell,
#             decoder_hidden,
#             decoder_cell,
#             attention_weights,
#             attention_weights_cum,
#             attention_context,
#             processed_memory,
#         ) = self.initialize_decoder_states(memory, mask=~get_mask_from_lengths(memory_lengths))

#         mel_lengths = torch.zeros(
#             [memory.size(0)], dtype=torch.int32, device=memory.device
#         )
#         not_finished = torch.ones(
#             [memory.size(0)], dtype=torch.int32, device=memory.device
#         )

#         mel_outputs, gate_outputs, alignments = (
#             torch.zeros(1),
#             torch.zeros(1),
#             torch.zeros(1),
#         )
#         first_iter = True
#         while True:
#             decoder_input = self.prenet(decoder_input)
#             (
#                 mel_output,
#                 gate_output,
#                 attention_hidden,
#                 attention_cell,
#                 decoder_hidden,
#                 decoder_cell,
#                 attention_weights,
#                 attention_weights_cum,
#                 attention_context,
#             ) = self.decode(
#                 decoder_input,
#                 attention_hidden,
#                 attention_cell,
#                 decoder_hidden,
#                 decoder_cell,
#                 attention_weights,
#                 attention_weights_cum,
#                 attention_context,
#                 memory,
#                 processed_memory,
#                 mask,
#             )

#             if first_iter:
#                 mel_outputs = mel_output.unsqueeze(0)
#                 gate_outputs = gate_output
#                 alignments = attention_weights
#                 first_iter = False
#             else:
#                 mel_outputs = torch.cat((mel_outputs, mel_output.unsqueeze(0)), dim=0)
#                 gate_outputs = torch.cat((gate_outputs, gate_output), dim=0)
#                 alignments = torch.cat((alignments, attention_weights), dim=0)

#             dec = (
#                 torch.le(torch.sigmoid(gate_output), self.gate_threshold)
#                 .to(torch.int32)
#                 .squeeze(1)
#             )

#             not_finished = not_finished * dec
#             mel_lengths += not_finished

#             if self.early_stopping and torch.sum(not_finished) == 0:
#                 break
#             if len(mel_outputs) == self.max_decoder_steps:
#                 print("Warning! Reached max decoder steps")
#                 break

#             decoder_input = mel_output

#         mel_outputs, gate_outputs, alignments = self.parse_decoder_outputs(
#             mel_outputs, gate_outputs, alignments
#         )

#         return mel_outputs, gate_outputs, alignments, mel_lengths

#     def inference_noattention(self, memory, attention_map):
#         """Decoder inference
#         PARAMS
#         ------
#         memory: Encoder outputs

#         RETURNS
#         -------
#         mel_outputs: mel outputs from the decoder
#         gate_outputs: gate outputs from the decoder
#         alignments: sequence of attention weights from the decoder
#         """
#         decoder_input = self.get_go_frame(memory)

#         self.initialize_decoder_states(memory, mask=None)

#         B = memory.size(0)
#         mel_outputs = torch.empty(
#             B, 0, self.n_frames_per_step_current * self.n_mel_channels
#         )
#         if torch.cuda.is_available():
#             mel_outputs = mel_outputs.cuda()
#         gate_outputs, alignments = [], []
#         for i in range(len(attention_map)):

#             attention = attention_map[i]
#             decoder_input = torch.cat((self.prenet(decoder_input)), dim=1)
#             mel_output, gate_output, alignment = self.decode(decoder_input, attention)
#             mel_output, gate_output, alignment = self.decode(decoder_input)
#             mel_output = mel_output[
#                 :, 0 : self.n_mel_channels * self.n_frames_per_step_current
#             ].unsqueeze(1)

#             mel_outputs = torch.cat([mel_outputs, mel_output], dim=1)
#             gate_outputs += [gate_output.squeeze()] * self.n_frames_per_step_current
#             alignments += [alignment]

#             decoder_input = mel_output[:, -1, -1 * self.n_mel_channels :]

#         mel_outputs, gate_outputs, alignments = self.parse_decoder_outputs(
#             mel_outputs, gate_outputs, alignments
#         )

#         return mel_outputs, gate_outputs, alignments

In [2]:
# #mix v2
# class Decoder(nn.Module):

#     def __init__(self, hparams):
#         super(Decoder, self).__init__()
#         self.n_mel_channels = hparams.n_mel_channels
#         self.n_frames_per_step_initial = hparams.n_frames_per_step_initial
#         self.n_frames_per_step_current = hparams.n_frames_per_step_initial
#         self.encoder_embedding_dim = hparams.encoder_embedding_dim
#         self.attention_rnn_dim = hparams.attention_rnn_dim
#         self.decoder_rnn_dim = hparams.decoder_rnn_dim
#         self.prenet_dim = hparams.prenet_dim
#         self.max_decoder_steps = hparams.max_decoder_steps
#         self.gate_threshold = hparams.gate_threshold
#         self.p_attention_dropout = hparams.p_attention_dropout
#         self.p_decoder_dropout = hparams.p_decoder_dropout

#         self.prenet = Prenet(
#             hparams.n_mel_channels,
#             [hparams.prenet_dim, hparams.prenet_dim],
#         )

#         self.attention_rnn = nn.LSTMCell(
#             hparams.prenet_dim + hparams.encoder_embedding_dim,
#             hparams.attention_rnn_dim)

#         self.attention_layer = Attention(
#             hparams.attention_rnn_dim, hparams.encoder_embedding_dim,
#             hparams.attention_dim, hparams.attention_location_n_filters,
#             hparams.attention_location_kernel_size,fp16_run=hparams.fp16_run)

#         self.decoder_rnn = nn.LSTMCell(
#             hparams.attention_rnn_dim + hparams.encoder_embedding_dim,
#             hparams.decoder_rnn_dim, 1)

#         self.linear_projection = LinearNorm(
#             hparams.decoder_rnn_dim + self.encoder_embedding_dim,
#             hparams.n_mel_channels * hparams.n_frames_per_step_initial,
#         )

#         self.gate_layer = LinearNorm(
#             hparams.decoder_rnn_dim + hparams.encoder_embedding_dim, 1,
#             bias=True, w_init_gain='sigmoid')


#     def get_go_frame(self, memory):
#         """Gets all zeros frames to use as first decoder input
#         PARAMS
#         ------
#         memory: decoder outputs
#         RETURNS
#         -------
#         decoder_input: all zeros frames
#         """
#         B = memory.size(0)
#         dtype = memory.dtype
#         device = memory.device
#         decoder_input = torch.zeros(
#             B, self.n_mel_channels * self.n_frames_per_step_initial, dtype=dtype, device=device
#         )
#         return decoder_input

# #     def get_go_frame(self, memory):
# #         """Gets all zeros frames to use as first decoder input
# #         PARAMS
# #         ------
# #         memory: decoder outputs

# #         RETURNS
# #         -------
# #         decoder_input: all zeros frames
# #         """
# #         B = memory.size(0)
# #         decoder_input = Variable(memory.data.new(B, self.n_mel_channels).zero_())
# #         return decoder_input

#     def initialize_decoder_states(self, memory):
#         """Initializes attention rnn states, decoder rnn states, attention
#         weights, attention cumulative weights, attention context, stores memory
#         and stores processed memory
#         PARAMS
#         ------
#         memory: Encoder outputs
#         mask: Mask for padded data if training, expects None for inference
#         """
#         B = memory.size(0)
#         MAX_TIME = memory.size(1)
#         dtype = memory.dtype
#         device = memory.device

#         attention_hidden = torch.zeros(
#             B, self.attention_rnn_dim, dtype=dtype, device=device
#         )
#         attention_cell = torch.zeros(
#             B, self.attention_rnn_dim, dtype=dtype, device=device
#         )

#         decoder_hidden = torch.zeros(
#             B, self.decoder_rnn_dim, dtype=dtype, device=device
#         )
#         decoder_cell = torch.zeros(B, self.decoder_rnn_dim, dtype=dtype, device=device)

#         attention_weights = torch.zeros(B, MAX_TIME, dtype=dtype, device=device)
#         attention_weights_cum = torch.zeros(B, MAX_TIME, dtype=dtype, device=device)
#         attention_context = torch.zeros(
#             B, self.encoder_embedding_dim, dtype=dtype, device=device
#         )

#         processed_memory = self.attention_layer.memory_layer(memory)

#         return (
#             attention_hidden,
#             attention_cell,
#             decoder_hidden,
#             decoder_cell,
#             attention_weights,
#             attention_weights_cum,
#             attention_context,
#             processed_memory,
#         )

#     def parse_decoder_inputs(self, decoder_inputs):
#         """Prepares decoder inputs, i.e. mel outputs
#         PARAMS
#         ------
#         decoder_inputs: inputs used for teacher-forced training, i.e. mel-specs
#         RETURNS
#         -------
#         inputs: processed decoder inputs
#         """
#         # (B, n_mel_channels, T_out) -> (B, T_out, n_mel_channels)
#         decoder_inputs = decoder_inputs.transpose(1, 2)
#         decoder_inputs = decoder_inputs.view(
#             decoder_inputs.size(0),
#             int(decoder_inputs.size(1) / self.n_frames_per_step),
#             -1,
#         )
#         # (B, T_out, n_mel_channels) -> (T_out, B, n_mel_channels)
#         decoder_inputs = decoder_inputs.transpose(0, 1)
#         return decoder_inputs

#     def parse_decoder_outputs(self, mel_outputs, gate_outputs, alignments):
#         """Prepares decoder outputs for output
#         PARAMS
#         ------
#         mel_outputs:
#         gate_outputs: gate output energies
#         alignments:
#         RETURNS
#         -------
#         mel_outputs:
#         gate_outpust: gate output energies
#         alignments:
#         """
#         # (T_out, B) -> (B, T_out)
#         alignments = alignments.transpose(0, 1).contiguous()
#         # (T_out, B) -> (B, T_out)
#         gate_outputs = gate_outputs.transpose(0, 1).contiguous()
#         # (T_out, B, n_mel_channels) -> (B, T_out, n_mel_channels)
#         mel_outputs = mel_outputs.transpose(0, 1).contiguous()
#         # decouple frames per step
#         shape = (mel_outputs.shape[0], -1, self.n_mel_channels)
#         mel_outputs = mel_outputs.view(*shape)
#         # (B, T_out, n_mel_channels) -> (B, n_mel_channels, T_out)
#         mel_outputs = mel_outputs.transpose(1, 2)

#         return mel_outputs, gate_outputs, alignments

#     def decode(
#         self,
#         decoder_input,
#         attention_hidden,
#         attention_cell,
#         decoder_hidden,
#         decoder_cell,
#         attention_weights,
#         attention_weights_cum,
#         attention_context,
#         memory,
#         processed_memory,
#         mask,
#     ):
#         """Decoder step using stored states, attention and memory
#         PARAMS
#         ------
#         decoder_input: previous mel output
#         RETURNS
#         -------
#         mel_output:
#         gate_output: gate output energies
#         attention_weights:
#         """
#         cell_input = torch.cat((decoder_input, attention_context), -1)

#         attention_hidden, attention_cell = self.attention_rnn(
#             cell_input, (attention_hidden, attention_cell)
#         )
#         attention_hidden = F.dropout(
#             attention_hidden, self.p_attention_dropout, self.training
#         )

#         attention_weights_cat = torch.cat(
#             (attention_weights.unsqueeze(1), attention_weights_cum.unsqueeze(1)), dim=1
#         )
#         attention_context, attention_weights = self.attention_layer(
#             attention_hidden, memory, processed_memory, attention_weights_cat, mask
#         )

#         attention_weights_cum += attention_weights
#         decoder_input = torch.cat((attention_hidden, attention_context), -1)

#         decoder_hidden, decoder_cell = self.decoder_rnn(
#             decoder_input, (decoder_hidden, decoder_cell)
#         )
#         decoder_hidden = F.dropout(
#             decoder_hidden, self.p_decoder_dropout, self.training
#         )

#         decoder_hidden_attention_context = torch.cat(
#             (decoder_hidden, attention_context), dim=1
#         )
#         decoder_output = self.linear_projection(decoder_hidden_attention_context)

#         gate_prediction = self.gate_layer(decoder_hidden_attention_context)

#         return (
#             decoder_output,
#             gate_prediction,
#             attention_hidden,
#             attention_cell,
#             decoder_hidden,
#             decoder_cell,
#             attention_weights,
#             attention_weights_cum,
#             attention_context,
#         )

#     @torch.jit.ignore
#     def forward(self, memory, decoder_inputs, memory_lengths):
#         """Decoder forward pass for training
#         PARAMS
#         ------
#         memory: Encoder outputs
#         decoder_inputs: Decoder inputs for teacher forcing. i.e. mel-specs
#         memory_lengths: Encoder output lengths for attention masking.
#         RETURNS
#         -------
#         mel_outputs: mel outputs from the decoder
#         gate_outputs: gate outputs from the decoder
#         alignments: sequence of attention weights from the decoder
#         """

#         decoder_input = self.get_go_frame(memory).unsqueeze(0)
#         decoder_inputs = self.parse_decoder_inputs(decoder_inputs)
#         decoder_inputs = torch.cat((decoder_input, decoder_inputs), dim=0)
#         decoder_inputs = self.prenet(decoder_inputs)

#         mask = get_mask_from_lengths(memory_lengths)
#         (
#             attention_hidden,
#             attention_cell,
#             decoder_hidden,
#             decoder_cell,
#             attention_weights,
#             attention_weights_cum,
#             attention_context,
#             processed_memory,
#         ) = self.initialize_decoder_states(memory)

#         mel_outputs, gate_outputs, alignments = [], [], []
#         while len(mel_outputs) < decoder_inputs.size(0) - 1:
#             decoder_input = decoder_inputs[len(mel_outputs)]
#             (
#                 mel_output,
#                 gate_output,
#                 attention_hidden,
#                 attention_cell,
#                 decoder_hidden,
#                 decoder_cell,
#                 attention_weights,
#                 attention_weights_cum,
#                 attention_context,
#             ) = self.decode(
#                 decoder_input,
#                 attention_hidden,
#                 attention_cell,
#                 decoder_hidden,
#                 decoder_cell,
#                 attention_weights,
#                 attention_weights_cum,
#                 attention_context,
#                 memory,
#                 processed_memory,
#                 mask,
#             )

#             mel_outputs += [mel_output.squeeze(1)]
#             gate_outputs += [gate_output.squeeze(1)]
#             alignments += [attention_weights]

#         mel_outputs, gate_outputs, alignments = self.parse_decoder_outputs(
#             torch.stack(mel_outputs), torch.stack(gate_outputs), torch.stack(alignments)
#         )

#         return mel_outputs, gate_outputs, alignments

#     def inference(self, memory, memory_lengths):
#         """Decoder inference
#         PARAMS
#         ------
#         memory: Encoder outputs
#         RETURNS
#         -------
#         mel_outputs: mel outputs from the decoder
#         gate_outputs: gate outputs from the decoder
#         alignments: sequence of attention weights from the decoder
#         """
#         decoder_input = self.get_go_frame(memory)

#         mask = get_mask_from_lengths(memory_lengths)
#         (
#             attention_hidden,
#             attention_cell,
#             decoder_hidden,
#             decoder_cell,
#             attention_weights,
#             attention_weights_cum,
#             attention_context,
#             processed_memory,
#         ) = self.initialize_decoder_states(memory)

#         mel_lengths = torch.zeros(
#             [memory.size(0)], dtype=torch.int32, device=memory.device
#         )
#         not_finished = torch.ones(
#             [memory.size(0)], dtype=torch.int32, device=memory.device
#         )

#         mel_outputs, gate_outputs, alignments = (
#             torch.zeros(1),
#             torch.zeros(1),
#             torch.zeros(1),
#         )
#         first_iter = True
#         while True:
#             decoder_input = self.prenet(decoder_input)
#             (
#                 mel_output,
#                 gate_output,
#                 attention_hidden,
#                 attention_cell,
#                 decoder_hidden,
#                 decoder_cell,
#                 attention_weights,
#                 attention_weights_cum,
#                 attention_context,
#             ) = self.decode(
#                 decoder_input,
#                 attention_hidden,
#                 attention_cell,
#                 decoder_hidden,
#                 decoder_cell,
#                 attention_weights,
#                 attention_weights_cum,
#                 attention_context,
#                 memory,
#                 processed_memory,
#                 mask,
#             )

#             if first_iter:
#                 mel_outputs = mel_output.unsqueeze(0)
#                 gate_outputs = gate_output
#                 alignments = attention_weights
#                 first_iter = False
#             else:
#                 mel_outputs = torch.cat((mel_outputs, mel_output.unsqueeze(0)), dim=0)
#                 gate_outputs = torch.cat((gate_outputs, gate_output), dim=0)
#                 alignments = torch.cat((alignments, attention_weights), dim=0)

#             dec = (
#                 torch.le(torch.sigmoid(gate_output), self.gate_threshold)
#                 .to(torch.int32)
#                 .squeeze(1)
#             )

#             not_finished = not_finished * dec
#             mel_lengths += not_finished

# #             if self.early_stopping and torch.sum(not_finished) == 0:
# #                 break
#             if len(mel_outputs) == self.max_decoder_steps:
#                 print("Warning! Reached max decoder steps")
#                 break

#             decoder_input = mel_output

#         mel_outputs, gate_outputs, alignments = self.parse_decoder_outputs(
#             mel_outputs, gate_outputs, alignments
#         )

#         return mel_outputs, gate_outputs, alignments, mel_lengths

In [3]:
# mix v3
from torch import nn
from uberduck_ml_dev.models.base import TTSModel
from uberduck_ml_dev.models.common import Attention, Conv1d, LinearNorm, GST
from uberduck_ml_dev.text.symbols import symbols
from uberduck_ml_dev.vendor.tfcompat.hparam import HParams
from uberduck_ml_dev.utils.utils import to_gpu, get_mask_from_lengths
import numpy as np
import torch
from torch.autograd import Variable
from torch.cuda.amp import autocast
from torch.nn import functional as F


class Decoder(nn.Module):
    def __init__(self, hparams):
        super().__init__()
        self.n_mel_channels = hparams.n_mel_channels
        self.n_frames_per_step_initial = hparams.n_frames_per_step_initial
        self.n_frames_per_step_current = hparams.n_frames_per_step_initial
        self.encoder_embedding_dim = hparams.encoder_embedding_dim
        self.attention_rnn_dim = hparams.attention_rnn_dim
        self.decoder_rnn_dim = hparams.decoder_rnn_dim
        self.prenet_dim = hparams.prenet_dim
        self.max_decoder_steps = hparams.max_decoder_steps
        self.gate_threshold = hparams.gate_threshold
        self.p_attention_dropout = hparams.p_attention_dropout
        self.p_decoder_dropout = hparams.p_decoder_dropout
        self.p_teacher_forcing = hparams.p_teacher_forcing

        self.prenet = Prenet(
            hparams.n_mel_channels,
            [hparams.prenet_dim, hparams.prenet_dim],
        )

        self.attention_rnn = nn.LSTMCell(
            hparams.prenet_dim + self.encoder_embedding_dim,
            hparams.attention_rnn_dim,
        )

        self.attention_layer = Attention(
            hparams.attention_rnn_dim,
            self.encoder_embedding_dim,
            hparams.attention_dim,
            hparams.attention_location_n_filters,
            hparams.attention_location_kernel_size,
            fp16_run=hparams.fp16_run,
        )

        self.decoder_rnn = nn.LSTMCell(
            hparams.attention_rnn_dim + self.encoder_embedding_dim,
            hparams.decoder_rnn_dim,
            1,
        )

        self.linear_projection = LinearNorm(
            hparams.decoder_rnn_dim + self.encoder_embedding_dim,
            hparams.n_mel_channels * hparams.n_frames_per_step_initial,
        )

        self.gate_layer = LinearNorm(
            hparams.decoder_rnn_dim + self.encoder_embedding_dim,
            1,
            bias=True,
            w_init_gain="sigmoid",
        )

    def set_current_frames_per_step(self, n_frames: int):
        self.n_frames_per_step_current = n_frames

    def get_go_frame(self, memory):
        """Gets all zeros frames to use as first decoder input
        PARAMS
        ------
        memory: decoder outputs

        RETURNS
        -------
        decoder_input: all zeros frames
        """
        B = memory.size(0)
        decoder_input = Variable(memory.data.new(B, self.n_mel_channels).zero_())
        return decoder_input

    def initialize_decoder_states(self, memory, mask):
        """Initializes attention rnn states, decoder rnn states, attention
        weights, attention cumulative weights, attention context, stores memory
        and stores processed memory
        PARAMS
        ------
        memory: Encoder outputs
        mask: Mask for padded data if training, expects None for inference
        """
        B = memory.size(0)
        MAX_TIME = memory.size(1)

        self.attention_hidden = Variable(
            memory.data.new(B, self.attention_rnn_dim).zero_()
        )
        self.attention_cell = Variable(
            memory.data.new(B, self.attention_rnn_dim).zero_()
        )

        self.decoder_hidden = Variable(memory.data.new(B, self.decoder_rnn_dim).zero_())
        self.decoder_cell = Variable(memory.data.new(B, self.decoder_rnn_dim).zero_())

        self.attention_weights = Variable(memory.data.new(B, MAX_TIME).zero_())
        self.attention_weights_cum = Variable(memory.data.new(B, MAX_TIME).zero_())
        self.attention_context = Variable(
            memory.data.new(B, self.encoder_embedding_dim).zero_()
        )

        self.memory = memory
        self.processed_memory = self.attention_layer.memory_layer(memory)
        self.mask = mask

    def parse_decoder_inputs(self, decoder_inputs):
        """Prepares decoder inputs, i.e. mel outputs
        PARAMS
        ------
        decoder_inputs: inputs used for teacher-forced training, i.e. mel-specs

        RETURNS
        -------
        inputs: processed decoder inputs

        """
        # (B, n_mel_channels, T_out) -> (B, T_out, n_mel_channels)

        decoder_inputs = decoder_inputs.transpose(1, 2)
        decoder_inputs = decoder_inputs.contiguous()
        # print(decoder_inputs.shape, 'dec_in')
        # print(decoder_inputs.size(0), int(decoder_inputs.size(1)), self.n_frames_per_step_current)
        decoder_inputs = decoder_inputs.view(
            decoder_inputs.size(0),
            int(decoder_inputs.size(1) / self.n_frames_per_step_current),
            -1,
        )
        # (B, T_out, n_mel_channels) -> (T_out, B, n_mel_channels)
        decoder_inputs = decoder_inputs.transpose(0, 1)
        return decoder_inputs

    def parse_decoder_outputs(self, mel_outputs, gate_outputs, alignments):
        """Prepares decoder outputs for output
        PARAMS
        ------
        mel_outputs:
        gate_outputs: gate output energies
        alignments:

        RETURNS
        -------
        mel_outputs:
        gate_outpust: gate output energies
        alignments:
        """
        # (T_out, B) -> (B, T_out)
        alignments = torch.stack(alignments).transpose(0, 1)
        # (T_out, B) -> (B, T_out)
        gate_outputs = torch.stack(gate_outputs)
        if len(gate_outputs.size()) > 1:
            gate_outputs = gate_outputs.transpose(0, 1)
        else:
            gate_outputs = gate_outputs[None]
        gate_outputs = gate_outputs.contiguous()
        # (B, T_out, n_mel_channels * n_frames_per_step) -> (B, T_out * n_frames_per_step, n_mel_channels)
        mel_outputs = mel_outputs.view(mel_outputs.size(0), -1, self.n_mel_channels)
        # (B, T_out, n_mel_channels) -> (B, n_mel_channels, T_out)
        mel_outputs = mel_outputs.transpose(1, 2)

        return mel_outputs, gate_outputs, alignments

    def decode(self, decoder_input, attention_weights=None):
        """Decoder step using stored states, attention and memory
        PARAMS
        ------
        decoder_input: previous mel output

        RETURNS
        -------
        mel_output:
        gate_output: gate output energies
        attention_weights:
        """
        cell_input = torch.cat((decoder_input, self.attention_context), -1)
        self.attention_hidden, self.attention_cell = self.attention_rnn(
            cell_input, (self.attention_hidden, self.attention_cell)
        )
        self.attention_hidden = F.dropout(
            self.attention_hidden, self.p_attention_dropout, self.training
        )
        self.attention_cell = F.dropout(
            self.attention_cell, self.p_attention_dropout, self.training
        )

        attention_weights_cat = torch.cat(
            (
                self.attention_weights.unsqueeze(1),
                self.attention_weights_cum.unsqueeze(1),
            ),
            dim=1,
        )
        self.attention_context, self.attention_weights = self.attention_layer(
            self.attention_hidden,
            self.memory,
            self.processed_memory,
            attention_weights_cat,
            self.mask,
            attention_weights,
        )

        self.attention_weights_cum += self.attention_weights
        decoder_input = torch.cat((self.attention_hidden, self.attention_context), -1)
        self.decoder_hidden, self.decoder_cell = self.decoder_rnn(
            decoder_input, (self.decoder_hidden, self.decoder_cell)
        )
        self.decoder_hidden = F.dropout(
            self.decoder_hidden, self.p_decoder_dropout, self.training
        )
        self.decoder_cell = F.dropout(
            self.decoder_cell, self.p_decoder_dropout, self.training
        )

        decoder_hidden_attention_context = torch.cat(
            (self.decoder_hidden, self.attention_context), dim=1
        )

        decoder_output = self.linear_projection(decoder_hidden_attention_context)

        gate_prediction = self.gate_layer(decoder_hidden_attention_context)
        return decoder_output, gate_prediction, self.attention_weights

    def forward(self, memory, decoder_inputs, memory_lengths):
        """Decoder forward pass for training
        PARAMS
        ------
        memory: Encoder outputs
        decoder_inputs: Decoder inputs for teacher forcing. i.e. mel-specs
        memory_lengths: Encoder output lengths for attention masking.

        RETURNS
        -------
        mel_outputs: mel outputs from the decoder
        gate_outputs: gate outputs from the decoder
        alignments: sequence of attention weights from the decoder
        """
        B = memory.size(0)
        decoder_inputs = self.parse_decoder_inputs(decoder_inputs)
        decoder_inputs = decoder_inputs.reshape(
            -1, decoder_inputs.size(1), self.n_mel_channels
        )
        decoder_input = self.get_go_frame(memory).unsqueeze(0)
        decoder_inputs = torch.cat((decoder_input, decoder_inputs), dim=0)
        decoder_inputs = self.prenet(decoder_inputs)

        self.initialize_decoder_states(
            memory, mask=~get_mask_from_lengths(memory_lengths)
        )

        mel_outputs = torch.empty(
            B, 0, self.n_frames_per_step_current * self.n_mel_channels
        )
        if torch.cuda.is_available():
            mel_outputs = mel_outputs.cuda()
        gate_outputs, alignments = [], []
        desired_output_frames = decoder_inputs.size(0) / self.n_frames_per_step_current
        while mel_outputs.size(1) < desired_output_frames - 1:
            if (
                mel_outputs.size(1) == 0
                or np.random.uniform(0.0, 1.0) <= self.p_teacher_forcing
            ):
                teacher_forced_frame = decoder_inputs[
                    mel_outputs.size(1) * self.n_frames_per_step_current
                ]

                to_concat = (teacher_forced_frame,)
                decoder_input = torch.cat(to_concat, dim=1)
            else:

                # NOTE(zach): we may need to concat these as we go to ensure that
                # it's easy to retrieve the last n_frames_per_step_init frames.
                to_concat = (
                    self.prenet(
                        mel_outputs[:, -1, -1 * self.n_frames_per_step_current :]
                    ),
                )
                decoder_input = torch.cat(to_concat, dim=1)
            # NOTE(zach): When training with fp16_run == True, decoder_rnn seems to run into
            # issues with NaNs in gradient, maybe due to vanishing gradients.
            # Disable half-precision for this call to work around the issue.
            with autocast(enabled=False):
                mel_output, gate_output, attention_weights = self.decode(decoder_input)
            mel_outputs = torch.cat(
                [
                    mel_outputs,
                    mel_output[
                        :, 0 : self.n_mel_channels * self.n_frames_per_step_current
                    ].unsqueeze(1),
                ],
                dim=1,
            )
            gate_outputs += [gate_output.squeeze()] * self.n_frames_per_step_current
            alignments += [attention_weights]

        mel_outputs, gate_outputs, alignments = self.parse_decoder_outputs(
            mel_outputs, gate_outputs, alignments
        )

        return mel_outputs, gate_outputs, alignments

    def inference(self, memory, memory_lengths):
        """Decoder inference
        PARAMS
        ------
        memory: Encoder outputs

        RETURNS
        -------
        mel_outputs: mel outputs from the decoder
        gate_outputs: gate outputs from the decoder
        alignments: sequence of attention weights from the decoder
        """
        # pdb.set_trace()
        decoder_input = self.get_go_frame(memory)
        self.initialize_decoder_states(
            memory, mask=~get_mask_from_lengths(memory_lengths)
        )

        B = memory.size(0)
        mel_outputs = torch.empty(
            B, 0, self.n_frames_per_step_current * self.n_mel_channels
        )
        if torch.cuda.is_available():
            mel_outputs = mel_outputs.cuda()
        gate_outputs, alignments = [], []

        mel_lengths = torch.zeros(
            [memory.size(0)], dtype=torch.int32, device=memory.device
        )
        not_finished = torch.ones(
            [memory.size(0)], dtype=torch.int32, device=memory.device
        )

        while True:

            to_cat = (self.prenet(decoder_input),)

            decoder_input = torch.cat(to_cat, dim=1)
            mel_output, gate_output, alignment = self.decode(decoder_input)
            mel_output = mel_output[
                :, 0 : self.n_mel_channels * self.n_frames_per_step_current
            ].unsqueeze(1)

            mel_outputs = torch.cat([mel_outputs, mel_output], dim=1)
            gate_outputs += [gate_output.squeeze()] * self.n_frames_per_step_current
            alignments += [alignment]

            #             if torch.sigmoid(gate_output.data) > self.gate_threshold:
            #                 break
            dec = (
                torch.le(torch.sigmoid(gate_output), self.gate_threshold)
                .to(torch.int32)
                .squeeze(1)
            )

            not_finished = not_finished * dec
            mel_lengths += not_finished
            if torch.sum(not_finished) == 0:
                break
            if len(mel_outputs) == self.max_decoder_steps:
                print("Warning! Reached max decoder steps")
                break

            decoder_input = mel_output[:, -1, -1 * self.n_mel_channels :]

        mel_outputs, gate_outputs, alignments = self.parse_decoder_outputs(
            mel_outputs, gate_outputs, alignments
        )

        return mel_outputs, gate_outputs, alignments

    def inference_noattention(self, memory, attention_map):
        """Decoder inference
        PARAMS
        ------
        memory: Encoder outputs

        RETURNS
        -------
        mel_outputs: mel outputs from the decoder
        gate_outputs: gate outputs from the decoder
        alignments: sequence of attention weights from the decoder
        """
        decoder_input = self.get_go_frame(memory)

        self.initialize_decoder_states(memory, mask=None)

        B = memory.size(0)
        mel_outputs = torch.empty(
            B, 0, self.n_frames_per_step_current * self.n_mel_channels
        )
        if torch.cuda.is_available():
            mel_outputs = mel_outputs.cuda()
        gate_outputs, alignments = [], []
        for i in range(len(attention_map)):

            attention = attention_map[i]
            decoder_input = torch.cat((self.prenet(decoder_input)), dim=1)
            mel_output, gate_output, alignment = self.decode(decoder_input, attention)
            mel_output, gate_output, alignment = self.decode(decoder_input)
            mel_output = mel_output[
                :, 0 : self.n_mel_channels * self.n_frames_per_step_current
            ].unsqueeze(1)

            mel_outputs = torch.cat([mel_outputs, mel_output], dim=1)
            gate_outputs += [gate_output.squeeze()] * self.n_frames_per_step_current
            alignments += [alignment]

            decoder_input = mel_output[:, -1, -1 * self.n_mel_channels :]

        mel_outputs, gate_outputs, alignments = self.parse_decoder_outputs(
            mel_outputs, gate_outputs, alignments
        )

        return mel_outputs, gate_outputs, alignments

In [4]:
# ub_ml_dev
# class Decoder(nn.Module):
#     def __init__(self, hparams):
#         super(Decoder, self).__init__()
#         self.n_mel_channels = hparams.n_mel_channels
#         self.n_frames_per_step = hparams.n_frames_per_step
#         self.encoder_embedding_dim = hparams.encoder_embedding_dim
#         self.attention_rnn_dim = hparams.attention_rnn_dim
#         self.decoder_rnn_dim = hparams.decoder_rnn_dim
#         self.prenet_dim = hparams.prenet_dim
#         self.max_decoder_steps = hparams.max_decoder_steps
#         self.gate_threshold = hparams.gate_threshold
#         self.p_attention_dropout = hparams.p_attention_dropout
#         self.p_decoder_dropout = hparams.p_decoder_dropout

#         self.prenet = Prenet(
#             hparams.n_mel_channels * hparams.n_frames_per_step,
#             [hparams.prenet_dim, hparams.prenet_dim])

#         self.attention_rnn = nn.LSTMCell(
#             hparams.prenet_dim + hparams.encoder_embedding_dim,
#             hparams.attention_rnn_dim)

#         self.attention_layer = Attention(
#             hparams.attention_rnn_dim, hparams.encoder_embedding_dim,
#             hparams.attention_dim, hparams.attention_location_n_filters,
#             hparams.attention_location_kernel_size)

#         self.decoder_rnn = nn.LSTMCell(
#             hparams.attention_rnn_dim + hparams.encoder_embedding_dim,
#             hparams.decoder_rnn_dim, 1)

#         self.linear_projection = LinearNorm(
#             hparams.decoder_rnn_dim + hparams.encoder_embedding_dim,
#             hparams.n_mel_channels * hparams.n_frames_per_step)

#         self.gate_layer = LinearNorm(
#             hparams.decoder_rnn_dim + hparams.encoder_embedding_dim, 1,
#             bias=True, w_init_gain='sigmoid')

#     def get_go_frame(self, memory):
#         """ Gets all zeros frames to use as first decoder input
#         PARAMS
#         ------
#         memory: decoder outputs
#         RETURNS
#         -------
#         decoder_input: all zeros frames
#         """
#         B = memory.size(0)
#         decoder_input = Variable(memory.data.new(
#             B, self.n_mel_channels * self.n_frames_per_step).zero_())
#         return decoder_input

#     def initialize_decoder_states(self, memory, mask):
#         """ Initializes attention rnn states, decoder rnn states, attention
#         weights, attention cumulative weights, attention context, stores memory
#         and stores processed memory
#         PARAMS
#         ------
#         memory: Encoder outputs
#         mask: Mask for padded data if training, expects None for inference
#         """
#         B = memory.size(0)
#         MAX_TIME = memory.size(1)

#         self.attention_hidden = Variable(memory.data.new(
#             B, self.attention_rnn_dim).zero_())
#         self.attention_cell = Variable(memory.data.new(
#             B, self.attention_rnn_dim).zero_())

#         self.decoder_hidden = Variable(memory.data.new(
#             B, self.decoder_rnn_dim).zero_())
#         self.decoder_cell = Variable(memory.data.new(
#             B, self.decoder_rnn_dim).zero_())

#         self.attention_weights = Variable(memory.data.new(
#             B, MAX_TIME).zero_())
#         self.attention_weights_cum = Variable(memory.data.new(
#             B, MAX_TIME).zero_())
#         self.attention_context = Variable(memory.data.new(
#             B, self.encoder_embedding_dim).zero_())

#         self.memory = memory
#         self.processed_memory = self.attention_layer.memory_layer(memory)
#         self.mask = mask

#     def parse_decoder_inputs(self, decoder_inputs):
#         """ Prepares decoder inputs, i.e. mel outputs
#         PARAMS
#         ------
#         decoder_inputs: inputs used for teacher-forced training, i.e. mel-specs
#         RETURNS
#         -------
#         inputs: processed decoder inputs
#         """
#         # (B, n_mel_channels, T_out) -> (B, T_out, n_mel_channels)
#         decoder_inputs = decoder_inputs.transpose(1, 2)
#         decoder_inputs = decoder_inputs.view(
#             decoder_inputs.size(0),
#             int(decoder_inputs.size(1)/self.n_frames_per_step), -1)
#         # (B, T_out, n_mel_channels) -> (T_out, B, n_mel_channels)
#         decoder_inputs = decoder_inputs.transpose(0, 1)
#         return decoder_inputs

#     def parse_decoder_outputs(self, mel_outputs, gate_outputs, alignments):
#         """ Prepares decoder outputs for output
#         PARAMS
#         ------
#         mel_outputs:
#         gate_outputs: gate output energies
#         alignments:
#         RETURNS
#         -------
#         mel_outputs:
#         gate_outpust: gate output energies
#         alignments:
#         """
#         # (T_out, B) -> (B, T_out)
#         alignments = torch.stack(alignments).transpose(0, 1)
#         # (T_out, B) -> (B, T_out)
#         gate_outputs = torch.stack(gate_outputs).transpose(0, 1)
#         gate_outputs = gate_outputs.contiguous()
#         # (T_out, B, n_mel_channels) -> (B, T_out, n_mel_channels)
#         mel_outputs = torch.stack(mel_outputs).transpose(0, 1).contiguous()
#         # decouple frames per step
#         mel_outputs = mel_outputs.view(
#             mel_outputs.size(0), -1, self.n_mel_channels)
#         # (B, T_out, n_mel_channels) -> (B, n_mel_channels, T_out)
#         mel_outputs = mel_outputs.transpose(1, 2)

#         return mel_outputs, gate_outputs, alignments

#     def decode(self, decoder_input):
#         """ Decoder step using stored states, attention and memory
#         PARAMS
#         ------
#         decoder_input: previous mel output
#         RETURNS
#         -------
#         mel_output:
#         gate_output: gate output energies
#         attention_weights:
#         """
#         cell_input = torch.cat((decoder_input, self.attention_context), -1)
#         self.attention_hidden, self.attention_cell = self.attention_rnn(
#             cell_input, (self.attention_hidden, self.attention_cell))
#         self.attention_hidden = F.dropout(
#             self.attention_hidden, self.p_attention_dropout, self.training)

#         attention_weights_cat = torch.cat(
#             (self.attention_weights.unsqueeze(1),
#              self.attention_weights_cum.unsqueeze(1)), dim=1)
#         self.attention_context, self.attention_weights = self.attention_layer(
#             self.attention_hidden, self.memory, self.processed_memory,
#             attention_weights_cat, self.mask)

#         self.attention_weights_cum += self.attention_weights
#         decoder_input = torch.cat(
#             (self.attention_hidden, self.attention_context), -1)
#         self.decoder_hidden, self.decoder_cell = self.decoder_rnn(
#             decoder_input, (self.decoder_hidden, self.decoder_cell))
#         self.decoder_hidden = F.dropout(
#             self.decoder_hidden, self.p_decoder_dropout, self.training)

#         decoder_hidden_attention_context = torch.cat(
#             (self.decoder_hidden, self.attention_context), dim=1)
#         decoder_output = self.linear_projection(
#             decoder_hidden_attention_context)

#         gate_prediction = self.gate_layer(decoder_hidden_attention_context)
#         return decoder_output, gate_prediction, self.attention_weights

#     def forward(self, memory, decoder_inputs, memory_lengths):
#         """ Decoder forward pass for training
#         PARAMS
#         ------
#         memory: Encoder outputs
#         decoder_inputs: Decoder inputs for teacher forcing. i.e. mel-specs
#         memory_lengths: Encoder output lengths for attention masking.
#         RETURNS
#         -------
#         mel_outputs: mel outputs from the decoder
#         gate_outputs: gate outputs from the decoder
#         alignments: sequence of attention weights from the decoder
#         """

#         decoder_input = self.get_go_frame(memory).unsqueeze(0)
#         decoder_inputs = self.parse_decoder_inputs(decoder_inputs)
#         decoder_inputs = torch.cat((decoder_input, decoder_inputs), dim=0)
#         decoder_inputs = self.prenet(decoder_inputs)

#         self.initialize_decoder_states(
#             memory, mask=~get_mask_from_lengths(memory_lengths))

#         mel_outputs, gate_outputs, alignments = [], [], []
#         while len(mel_outputs) < decoder_inputs.size(0) - 1:
#             decoder_input = decoder_inputs[len(mel_outputs)]
#             mel_output, gate_output, attention_weights = self.decode(
#                 decoder_input)
#             mel_outputs += [mel_output.squeeze(1)]
#             gate_outputs += [gate_output.squeeze(1)]
#             alignments += [attention_weights]

#         mel_outputs, gate_outputs, alignments = self.parse_decoder_outputs(
#             mel_outputs, gate_outputs, alignments)

#         return mel_outputs, gate_outputs, alignments

#     def inference(self, memory):
#         """ Decoder inference
#         PARAMS
#         ------
#         memory: Encoder outputs
#         RETURNS
#         -------
#         mel_outputs: mel outputs from the decoder
#         gate_outputs: gate outputs from the decoder
#         alignments: sequence of attention weights from the decoder
#         """
#         decoder_input = self.get_go_frame(memory)

#         self.initialize_decoder_states(memory, mask=None)

#         mel_outputs, gate_outputs, alignments = [], [], []
#         while True:
#             decoder_input = self.prenet(decoder_input)
#             mel_output, gate_output, alignment = self.decode(decoder_input)

#             mel_outputs += [mel_output.squeeze(1)]
#             gate_outputs += [gate_output]
#             alignments += [alignment]

#             if torch.sigmoid(gate_output.data) > self.gate_threshold:
#                 break
#             elif len(mel_outputs) == self.max_decoder_steps:
#                 print("Warning! Reached max decoder steps")
#                 break

#             decoder_input = mel_output

#         mel_outputs, gate_outputs, alignments = self.parse_decoder_outputs(
#             mel_outputs, gate_outputs, alignments)

#         return mel_outputs, gate_outputs, alignments

In [5]:
# export


class Postnet(nn.Module):
    """Postnet
    - Five 1-d convolution with 512 channels and kernel size 5
    """

    def __init__(self, hparams):
        super(Postnet, self).__init__()
        self.dropout_rate = 0.5
        self.convolutions = nn.ModuleList()

        self.convolutions.append(
            nn.Sequential(
                Conv1d(
                    hparams.n_mel_channels,
                    hparams.postnet_embedding_dim,
                    kernel_size=hparams.postnet_kernel_size,
                    stride=1,
                    padding=int((hparams.postnet_kernel_size - 1) / 2),
                    dilation=1,
                    w_init_gain="tanh",
                ),
                nn.BatchNorm1d(hparams.postnet_embedding_dim),
            )
        )

        for i in range(1, hparams.postnet_n_convolutions - 1):
            self.convolutions.append(
                nn.Sequential(
                    Conv1d(
                        hparams.postnet_embedding_dim,
                        hparams.postnet_embedding_dim,
                        kernel_size=hparams.postnet_kernel_size,
                        stride=1,
                        padding=int((hparams.postnet_kernel_size - 1) / 2),
                        dilation=1,
                        w_init_gain="tanh",
                    ),
                    nn.BatchNorm1d(hparams.postnet_embedding_dim),
                )
            )

        self.convolutions.append(
            nn.Sequential(
                Conv1d(
                    hparams.postnet_embedding_dim,
                    hparams.n_mel_channels,
                    kernel_size=hparams.postnet_kernel_size,
                    stride=1,
                    padding=int((hparams.postnet_kernel_size - 1) / 2),
                    dilation=1,
                    w_init_gain="linear",
                ),
                nn.BatchNorm1d(hparams.n_mel_channels),
            )
        )

    def forward(self, x):
        for i in range(len(self.convolutions) - 1):
            x = F.dropout(
                torch.tanh(self.convolutions[i](x)), self.dropout_rate, self.training
            )
        x = F.dropout(self.convolutions[-1](x), self.dropout_rate, self.training)

        return x

In [6]:
# export


class Prenet(nn.Module):
    def __init__(self, in_dim, sizes):
        super().__init__()
        in_sizes = [in_dim] + sizes[:-1]
        self.layers = nn.ModuleList(
            [
                LinearNorm(in_size, out_size, bias=False)
                for (in_size, out_size) in zip(in_sizes, sizes)
            ]
        )
        self.dropout_rate = 0.5

    def forward(self, x):
        for linear in self.layers:
            x = F.dropout(F.relu(linear(x)), p=self.dropout_rate, training=True)
        return x

In [7]:
# export
class Encoder(nn.Module):
    """Encoder module:
    - Three 1-d convolution banks
    - Bidirectional LSTM
    """

    def __init__(self, hparams):
        super().__init__()

        convolutions = []
        for _ in range(hparams.encoder_n_convolutions):
            conv_layer = nn.Sequential(
                Conv1d(
                    hparams.encoder_embedding_dim,
                    hparams.encoder_embedding_dim,
                    kernel_size=hparams.encoder_kernel_size,
                    stride=1,
                    padding=int((hparams.encoder_kernel_size - 1) / 2),
                    dilation=1,
                    w_init_gain="relu",
                ),
                nn.BatchNorm1d(hparams.encoder_embedding_dim),
            )
            convolutions.append(conv_layer)
        self.convolutions = nn.ModuleList(convolutions)
        self.dropout_rate = 0.5

        self.lstm = nn.LSTM(
            hparams.encoder_embedding_dim,
            int(hparams.encoder_embedding_dim / 2),
            1,
            batch_first=True,
            bidirectional=True,
        )

    def forward(self, x, input_lengths):
        if x.size()[0] > 1:
            print("here")
            x_embedded = []
            for b_ind in range(x.size()[0]):  # TODO: Speed up
                curr_x = x[b_ind : b_ind + 1, :, : input_lengths[b_ind]].clone()
                for conv in self.convolutions:
                    curr_x = F.dropout(
                        F.relu(conv(curr_x)), self.dropout_rate, self.training
                    )
                x_embedded.append(curr_x[0].transpose(0, 1))
            x = torch.nn.utils.rnn.pad_sequence(x_embedded, batch_first=True)
        else:
            for conv in self.convolutions:
                x = F.dropout(F.relu(conv(x)), self.dropout_rate, self.training)
            x = x.transpose(1, 2)

        # pytorch tensor are not reversible, hence the conversion
        input_lengths = input_lengths.cpu().numpy()
        x = nn.utils.rnn.pack_padded_sequence(
            x, input_lengths, batch_first=True, enforce_sorted=False
        )

        self.lstm.flatten_parameters()
        outputs, _ = self.lstm(x)

        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs, batch_first=True)
        return outputs

    def inference(self, x, input_lengths):
        device = x.device
        for conv in self.convolutions:
            x = F.dropout(F.relu(conv(x.to(device))), 0.5, self.training)

        x = x.transpose(1, 2)

        input_lengths = input_lengths.cpu()
        x = nn.utils.rnn.pack_padded_sequence(
            x, input_lengths, batch_first=True, enforce_sorted=False
        )

        outputs, _ = self.lstm(x)

        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs, batch_first=True)

        return outputs

In [8]:
# export
class Tacotron2(TTSModel):
    def __init__(self, hparams):
        super().__init__(hparams)

        self.mask_padding = hparams.mask_padding
        self.fp16_run = hparams.fp16_run
        self.pos_weight = hparams.pos_weight
        self.n_mel_channels = hparams.n_mel_channels
        self.n_frames_per_step_initial = hparams.n_frames_per_step_initial
        self.n_frames_per_step_current = hparams.n_frames_per_step_initial
        self.embedding = nn.Embedding(self.n_symbols, hparams.symbols_embedding_dim)
        std = np.sqrt(2.0 / (self.n_symbols + hparams.symbols_embedding_dim))
        val = np.sqrt(3.0) * std  # uniform bounds for std
        self.embedding.weight.data.uniform_(-val, val)
        self.encoder = Encoder(hparams)
        self.decoder = Decoder(hparams)
        self.postnet = Postnet(hparams)

    def parse_batch(self, batch):
        (
            text_padded,
            input_lengths,
            mel_padded,
            gate_padded,
            output_lengths,
            speaker_ids,
            *_,
        ) = batch

        text_padded = to_gpu(text_padded).long()
        input_lengths = to_gpu(input_lengths).long()
        max_len = torch.max(input_lengths.data).item()
        mel_padded = to_gpu(mel_padded).float()
        gate_padded = to_gpu(gate_padded).float()
        output_lengths = to_gpu(output_lengths).long()
        ret_x = [
            text_padded,
            input_lengths,
            mel_padded,
            max_len,
            output_lengths,
            speaker_ids,
        ]
        return (
            tuple(ret_x),
            (mel_padded, gate_padded),
        )

    def parse_output(self, outputs, output_lengths=None):
        if self.mask_padding and output_lengths is not None:
            mask = ~get_mask_from_lengths(output_lengths)
            mask = mask.expand(self.n_mel_channels, mask.size(0), mask.size(1))
            mask = F.pad(mask, (0, outputs[0].size(2) - mask.size(2)))
            mask = mask.permute(1, 0, 2)

            outputs[0].data.masked_fill_(mask, 0.0)
            outputs[1].data.masked_fill_(mask, 0.0)
            outputs[2].data.masked_fill_(mask[:, 0, :], 1e3)  # gate energies

        return outputs

    def forward(self, inputs):
        (
            input_text,
            input_lengths,
            targets,
            max_len,
            output_lengths,
            *_,
        ) = inputs

        input_lengths, output_lengths = input_lengths.data, output_lengths.data

        embedded_inputs = self.embedding(input_text).transpose(1, 2)
        embedded_text = self.encoder(embedded_inputs, input_lengths)

        encoder_outputs = torch.cat((embedded_text,), dim=2)

        mel_outputs, gate_outputs, alignments = self.decoder(
            encoder_outputs, targets, memory_lengths=input_lengths
        )

        mel_outputs_postnet = self.postnet(mel_outputs)
        mel_outputs_postnet = mel_outputs + mel_outputs_postnet

        return self.parse_output(
            [mel_outputs, mel_outputs_postnet, gate_outputs, alignments], output_lengths
        )

    def inference(self, inputs):
        text, input_lengths, speaker_ids, *_ = inputs

        embedded_inputs = self.embedding(text).transpose(1, 2)
        embedded_text = self.encoder.inference(embedded_inputs, input_lengths)

        encoder_outputs = torch.cat((embedded_text,), dim=2)

        memory_lengths = input_lengths
        mel_outputs, gate_outputs, alignments = self.decoder.inference(
            encoder_outputs, memory_lengths
        )
        # mel_outputs, gate_outputs, alignments, mel_lengths = self.decoder.inference(encoder_outputs, memory_lengths)
        mel_outputs_postnet = self.postnet(mel_outputs)
        mel_outputs_postnet = mel_outputs + mel_outputs_postnet

        return self.parse_output(
            [mel_outputs, mel_outputs_postnet, gate_outputs, alignments]
        )

    def inference_noattention(self, inputs):
        """Run inference conditioned on an attention map.

        NOTE(zach): I don't think it is necessary to do a version
        of this without f0s passed as well, since it seems like we
        would always want to condition on pitch when conditioning on rhythm.
        """
        text, attention_map = inputs
        embedded_inputs = self.embedding(text).transpose(1, 2)
        embedded_text = self.encoder.inference(embedded_inputs)

        encoder_outputs = torch.cat((embedded_text,), dim=2)

        mel_outputs, gate_outputs, alignments = self.decoder.inference_noattention(
            encoder_outputs, attention_map
        )

        mel_outputs_postnet = self.postnet(mel_outputs)
        mel_outputs_postnet = mel_outputs + mel_outputs_postnet

        return self.parse_output(
            [mel_outputs, mel_outputs_postnet, gate_outputs, alignments]
        )

In [11]:
from uberduck_ml_dev.text.symbols import (
    DEFAULT_SYMBOLS,
    IPA_SYMBOLS,
    NVIDIA_TACO2_SYMBOLS,
    GRAD_TTS_SYMBOLS,
)


def pad_sequences(batch):
    input_lengths = torch.LongTensor([len(x) for x in batch])
    max_input_len = input_lengths.max()

    text_padded = torch.LongTensor(len(batch), max_input_len)
    text_padded.zero_()
    for i in range(len(batch)):
        text = batch[i]
        text_padded[i, : text.size(0)] = text

    return text_padded, input_lengths


def prepare_input_sequence(
    texts, cpu_run=False, arpabet=False, symbol_set=NVIDIA_TACO2_SYMBOLS
):
    p_arpabet = float(arpabet)
    seqs = []
    for text in texts:
        seqs.append(
            torch.IntTensor(
                text_to_sequence(
                    text,
                    ["english_cleaners"],
                    p_arpabet=p_arpabet,
                    symbol_set=symbol_set,
                )[:]
            )
        )
    text_padded, input_lengths = pad_sequences(seqs)
    if not cpu_run:
        text_padded = text_padded.cuda().long()
        input_lengths = input_lengths.cuda().long()
    else:
        text_padded = text_padded.long()
        input_lengths = input_lengths.long()

    return text_padded, input_lengths

In [12]:
config_file = "/mnt/disks/uberduck-experiments-v0/uberduck-ml-exp/configs/old/taco2_lj_wsnvidia.json"
from uberduck_ml_dev.models.tacotron2 import DEFAULTS as TACOTRON2_DEFAULTS
import json
from uberduck_ml_dev.text.util import text_to_sequence
from uberduck_ml_dev.vendor.tfcompat.hparam import HParams
import numpy as np
import torch
from uberduck_ml_dev.utils.audio import mel_to_audio
from uberduck_ml_dev.models.common import MelSTFT
import IPython.display as ipd

In [13]:
config = TACOTRON2_DEFAULTS.values()
with open(config_file) as f:
    config.update(json.load(f))
hparams = HParams(**config)
hparams.warm_start_name = (
    "/mnt/disks/uberduck-experiments-v0/models/tacotron2_statedict.pt"
)
symbol_set = "nvidia_taco2"
device = "cuda"
hparams.symbol_set = symbol_set
hparams.device = "cuda"
model = Tacotron2(hparams)
model = model.cuda()
model.from_pretrained(warm_start_path=hparams.warm_start_name, device=device)

In [14]:
import pdb

In [15]:
texts = ["hello i am sam", "Long beach compton."]
# texts = ["hello again its me sam im back", "Long beach compton."]
# speaker_ids = [0,0]
# texts = ["hello again its me sam"]
speaker_ids = [0, 0]

text_padded, input_lengths = prepare_input_sequence(
    texts, cpu_run=False, arpabet=False, symbol_set=symbol_set
)
input_ = text_padded, input_lengths, speaker_ids

# data0 = ["i am yes", 0]
# data1 = ["i am not", 0]
# to_infer = [data0, data1]
# to_infer_batch = Collate()._to_inference(to_infer)
# tacotron wants input = sequence, speakerid, length
output = model.inference(input_)

In [16]:
input_lengths

tensor([14, 19], device='cuda:0')

In [17]:
input_lengths

tensor([14, 19], device='cuda:0')

In [18]:
text_padded

tensor([[45, 42, 49, 49, 52, 11, 46, 11, 38, 50, 11, 56, 38, 50,  0,  0,  0,  0,
          0],
        [49, 52, 51, 44, 11, 39, 42, 38, 40, 45, 11, 40, 52, 50, 53, 57, 52, 51,
          7]], device='cuda:0')

In [20]:
post_mel = mel_to_audio(mel=output[0][1], algorithm="griffin-lim")
ipd.Audio(post_mel.numpy(), rate=22050)

In [126]:
output[1].shape

torch.Size([2, 80, 125])

In [30]:
self.decoder.inference(encoder_outputs, memory_lengths)

tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         False, False, False, False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True]])

In [ ]:
#these classes are under development
#make a class containing e.g. texts, sequences, that can be read as a batch in either forward passes and inference
#create lists for particular off-the-shelf models?

class TTSDataset(Dataset):
    
    def __init__(
        self,
        audiopaths_and_text: str,
        text_cleaners: List[str],
        p_arpabet: float,
        n_mel_channels: int,
        sample_rate: int,
        mel_fmin: float,
        mel_fmax: float,
        filter_length: int,
        hop_length: int,
        padding: int,
        win_length: int,
        symbol_set: str,
        max_wav_value: float = 32768.0,
        include_f0: bool = False,
        pos_weight: float = 10,
        f0_min: int = 80,
        f0_max: int = 880,
        harmonic_thresh=0.25,
        debug: bool = False,
        debug_dataset_size: int = None,
        oversample_weights=None,
        intersperse_text=False,
        intersperse_token=0,
    ):
        super().__init__()
        
        #oversample
        path = audiopaths_and_text
        oversample_weights = oversample_weights or {}
        self.audiopaths_and_text = oversample(
            load_filepaths_and_text(path), oversample_weights
        )
        
        #text to seq parameters
        self.text_cleaners = text_cleaners
        self.p_arpabet = p_arpabet
        self.intersperse_text = intersperse_text
        self.intersperse_token = intersperse_token
        
        self.stft = MelSTFT(
            filter_length=filter_length,
            hop_length=hop_length,
            win_length=win_length,
            n_mel_channels=n_mel_channels,
            sampling_rate=sample_rate,
            mel_fmin=mel_fmin,
            mel_fmax=mel_fmax,
            padding=padding,
        )
        self.max_wav_value = max_wav_value
        self.sample_rate = sample_rate
        self.filter_length = filter_length
        self.hop_length = hop_length
        self.mel_fmin = mel_fmin
        self.mel_fmax = mel_fmax
        self.include_f0 = include_f0
        self.f0_min = f0_min
        self.f0_max = f0_max
        self.harmonic_threshold = harmonic_thresh
        # speaker id lookup table
        speaker_ids = [i[2] for i in self.audiopaths_and_text]
        self.symbol_set = symbol_set


    def _get_f0(self, audio):
        f0, harmonic_rates, argmins, times = compute_yin(
            audio,
            self.sample_rate,
            self.filter_length,
            self.hop_length,
            self.f0_min,
            self.f0_max,
            self.harmonic_threshold,
        )
        pad = int((self.filter_length / self.hop_length) / 2)
        f0 = [0.0] * pad + f0 + [0.0] * pad
        f0 = np.array(f0, dtype=np.float32)
        return f0

    def _text_to_seq(self, audiopath_and_text):
        path, transcription, speaker_id = audiopath_and_text
        text_sequence = torch.LongTensor(
            text_to_sequence(
                transcription,
                self.text_cleaners,
                p_arpabet=self.p_arpabet,
                symbol_set=self.symbol_set,
            )
        )
        if self.intersperse_text:
            text_sequence = torch.LongTensor(
                intersperse(text_sequence.numpy(), self.intersperse_token)
            )  # add a blank token, whose id number is len(symbols)
        return(text_to_sequence)

       
    def _get_f0(self,audio):
        
        if self.include_f0:
        else:
            return None
    
    def _get_mel(self,):
        
    def _get_data(self, audiopath_and_text):

        sequence = self._text_to_seq(audiopath_and_text)
        audio = self._get_audio(audiopath_and_text)
        melspec = self._get_mel(audio)
        f0 = self._get_f0(audio)
        speaker_id = self._get_sid(audiopath_and_text)

        return (text_sequence, melspec, speaker_id, f0)
    
class Collate:
    '''
    Collate assembles batches from list indexed by sample id
    text, spectragram, etc'''
    def __init__(**args):
        
        n_frames_per_step: int = 1, 
        include_f0: bool = False,
        include_sid: bool = False,
        batch_format: str
            
    def _pad_sequence(self, batch):
        
        batch_size = len(batch)
        input_lengths = [len(x[0].shape[1]) for x in batch]
        max_input_len = input_lengths.max()
        text_padded = torch.LongTensor(batch_size, max_input_len)
        text_padded.zero_()
        for i in range(batch_size):
            text_padded[i, : batch[0][i].shape[1]] = batch[0][i]
            
        return(text_padded)

    def _pad_mel(self, batch):
        
        batch_size = len(batch)
        target_lengths = [len(x[0].shape[1]) for x in batch]
        max_target_len = max(target_len)
        textint_padded = torch.LongTensor(batch_size, max_input_len)
        textint_padded.zero_()
        for i in range(batch_size):
            textint_padded[i, : batch[0][i].shape[1]] = batch[0][i]
            
        #assert len(f0) = len(mel)
        return(text_padded)
    
    def _pad_f0(self, batch):
        return(None)
    
    def __call__(self, batch):

        text_padded, input_lengths = _pad_sequence(batch) #idx
        mel_padded, gate_padded, output_lengths = _pad_mel(batch)
        f0 = None
        batch = Batch(text = text_padded,
                     input_lengths = input_lengths,
                     mel_padded = mel_padded,
                     gate_padded = gate_padded,
                     output_lengths = output_lengths
                     f0 = f0,
                     speaker_ids = speaker_ids)

        if batch_format == 'taco2ss':
            return(text_padded, mel_padded, mel_padded, output_lengths, input_lengths)
        if batch_format == 'taco2ms':
            
        return(batch)  

    def inference(self, batch):
        
        if batch_format == 'taco2ss':
            return(self.text_padded, self.input_lengths)
        if batch_format == 'taco2ms':
            return(self.text_padded, self.input_lengths)
    #need to have pad_sequences equivalent
    def _to_tacotron2_singlespeaker_inference(self, batch):
        
        text_padded, input_lengths = _pad_sequence(batch_list) #idx
        mel_padded, gate_padded, output_lengths = _pad_mel(batch_list)        
        return(self.text_padded, self.input_lengths)

    # NOTE(zach): would model_inputs be better as a namedtuple or dataclass?
    def _to_mellotron_train_f0():
        
        batch = Batch
        return(
            text_padded,
            input_lengths,
            mel_padded,
            gate_padded,
            output_lengths,
            speaker_ids,
            f0_padded,
        )
    if self.include_f0:
        model_inputs = 
    else:
        model_inputs = (
            text_padded,
            input_lengths,
            mel_padded,
            gate_padded,
            output_lengths,
            speaker_ids,
        )


    def _to_tacotron2_multispeaker_inference(batch):

        input_lengths, ids_sorted_decreasing = torch.sort(
            torch.LongTensor([len(x) for x in batch]), dim=0, descending=True
        )
        max_input_len = input_lengths[0]

        text_padded = torch.LongTensor(len(batch), max_input_len)
        text_padded.zero_()
        for i in range(len(ids_sorted_decreasing)):
            text_padded[i, : text.size(0)] = batch[i]

        return(self.text_padded, self.speakers, self.input_lengths, self.sort_indices)
    
from dataclasses import dataclass

@dataclass
class Batch:
    textint_padded: torch.Tensor,
    input_lengths: list
    mel_padded: torch.Tensor
    gate_padded: 
    output_length: list,
    speaker_ids: list,
    f0_padded: list,

In [ ]:
config = TACOTRON2_DEFAULTS.values()
with open(config_file) as f:
    config.update(json.load(f))
hparams = HParams(**config)
hparams.warm_start_name = "/mnt/disks/uberduck-experiments-v0/models/taco2ljdefault"
model = Tacotron2(hparams)
model = model.cuda()
device = "cuda"
symbol_set = "nvidia_taco2"
model.from_pretrained(warm_start_path=hparams.warm_start_name, device=device)
# for inference
sequence1 = text_to_sequence(
    "You can see he is passionate about the company and what they are doing.",
    ["english_cleaners"],
    1.0,
    symbol_set=symbol_set,
)
sequence2 = text_to_sequence(
    "Long beach compton.", ["english_cleaners"], 1.0, symbol_set=symbol_set
)
# texts = ["hello i am sam."]#,"Long beach compton."]
texts = ["hello again its me sam", "Long beach compton."]
# print(torch.LongTensor(sequence)[None, :].shape)
sequence = torch.LongTensor(sequence1)[None].cuda()
input_ = [sequence, 0]
text_padded, input_lengths, sort_indices = prepare_input_sequence(
    texts, cpu_run=False, arpabet=True, symbol_set=symbol_set
)
speaker_ids = [0, 0]
input_ = text_padded, input_lengths, speaker_ids

data0 = ["i am yes", 0]
data1 = ["i am not", 0]
to_infer = [data0, data1]
# to_infer_batch = Collate()._to_inference(to_infer)

# tacotron wants input = sequence, speakerid, length
output = model.inference(input_)
output[0].shape
import torch

# input2 = torch.cat([input_[0],input_[0]], axis = 0)
# input2_ = [input2, [0,0]]
# output = model.inference(input2_)